**Sarcasm Target Detection Codebase**

In [ ]:
# Upload 'snippets.xlsx' from the local file system.
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving snippets.xlsx to snippets.xlsx
User uploaded file "snippets.xlsx" with length 28027 bytes


In [ ]:
# Mount google drive to use 'crawl-300d-2M.vec'. 
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# Import basic maths and processing libraries.
import pandas as pd
import numpy as np

# A RegexpTokenizer splits a string into substrings using a regular expression.
from nltk.tokenize import RegexpTokenizer 
from nltk.tokenize.treebank import TreebankWordDetokenizer

from keras.layers import Dense ,LSTM,concatenate,Input,Flatten
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

import pickle
import codecs

ModuleNotFoundError: ignored

In [ ]:
tokenizer = RegexpTokenizer(r'\w+') # try removing r
detokenizer = TreebankWordDetokenizer()
# Excel file contains no header column and names gives custom column names
df = pd.read_excel("snippets.xlsx", sheet_name=None, header=None, names=['Snippet','target'])
df = df['Sheet1']
embedding_file = '/gdrive/MyDrive/crawl-300d-2M.vec'

In [ ]:
def clearLine(line):
  cleanedLine = ''
  for letter in line:
    if letter == '?' or letter == '!' or letter == ',' or letter == '"' or letter == '-' or letter == ';' or letter == '.' or ord(letter) > 255 :
      cleanedLine += ' '
    else:
      cleanedLine += letter
  return cleanedLine

In [ ]:
def loadEmbed():
    print('loading word embeddings...')
    embeddings_index = {}
    f = codecs.open(embedding_file, encoding='utf-8')
    for line in f:
        # Line has the format : Word val1 val2 val3 ..... val300
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('found %s word vectors' % len(embeddings_index))
    return embeddings_index

In [ ]:
# Loading pre-trained embeddings
model=loadEmbed()

loading word embeddings...
found 1999996 word vectors


In [ ]:
# Stores all distinct words in the dataset.
all_words=[]

for i in range(len(df['Snippet'])):
    line = clearLine(df['Snippet'][i])
    all_words.extend(line.split())

# get all unique words from the dataset
all_words=list(dict.fromkeys(all_words))

# make all words to lowercase
all_words=[x.lower() for x in all_words]

In [ ]:
# Stores the vector representations of the words from the dataset.
embeddings={}
for each in all_words:
    # Get vectors for words in the dataset
    if each not in model.keys(): 
        embeddings[each]=model['unk']
    else:
        embeddings[each]=model[each]

embeddings['<pad>'] = [O]*300
embeddings['<start>'] = model["start"]
embeddings['<end>'] = model["end"]

In [ ]:
# Prepare the left context, right context, and candidate word from the dataset.
# Pre-Processed Data (ppd)

ppd = {}
ppd['left_context'] = []
ppd['right_context'] = []
ppd['Candidate_word'] = []
ppd['target_status'] = []

for i in range(len(df['Snippet'])):
  line = clearLine(df['Snippet'][i].lower());
  line = line.split()
  targetList = df['target'][i].lower().split(",")
  targetWord = []
  for tl in targetList:
    targetWord.extend(tl.split())
  
  for i in range(len(line)):
    word = line[i]
    ppd['Candidate_word'].append(word)
    ppd['left_context'].append(["<start>"] + line[:i])
    ppd['right_context'].append(line[i+1:] + ["<end>"])
    ppd['target_status'].append(int(word in targetWord))

In [ ]:
# Embedding Left Context
keras_left_context = []
for i in range(len(ppd['left_context'])):
    one_vector = []
    temp = ppd['left_context'][i]
    for m in temp:
        one_vector.append(embeddings[m])
    one_vector.extend([embeddings['<pad>'] for x in range(78 - len(ppd['left_context'][i]))])
    keras_left_context.append(one_vector)

In [ ]:
# Embedding Right Context 
keras_right_context = []
for i in range(len(ppd['right_context'])):
    one_vector = []
    temp = ppd['right_context'][i]
    for m in temp:
        one_vector.append(embeddings[m])
    one_vector.extend([embeddings['<pad>'] for x in range(78 - len(ppd['right_context'][i]))])
    keras_right_context.append(one_vector)

In [ ]:
# Embedding Candidate Word
keras_middle = []
for i in range(len(ppd['Candidate_words'])):
    keras_middle.append(embeddings[ppd['Candidate_words'][i]])

labels = ppd['target_status']

In [ ]:
#Saving the processed dataset in a pickle file
# f = open(b"Data_fast.pkl","wb")
# pickle.dump(zip(keras_left_context,keras_right_context,keras_middle,ppd['target_status']),f)

LIWC Trie Functions

In [ ]:
class LiwcTrieNode(object):
    def __init__(self, char: str):
        self.char = char
        self.word_finished = False
        self.children = []

def create_trie(liwc_dict):
	T = Liwc_Trie_Node()
	for category, words in liwc_dict.items():
		# print category, "starting"
		for word in words:
			insert_word(T, word, category)
			# print "\t", word.encode("utf-8"), "inserted"
		# print category, "done"
	return T

def insert_word(T, word, category):
	if word[len(word) - 1] != "*":
		word = word + "$"
	t = T
	i = 0
	while i < len(word):
		match = False
		for child in t.children:
			if child.character == word[i]:
				match = True
				t = child
				break
		if match == False:
			break
		else:
			i = i + 1
	while i < len(word):
		child = Liwc_Trie_Node()
		child.character = word[i]
		t.children.append(child)
		t = child
		i = i + 1
	t.categories.add(category)

def get_liwc_categories(T, word):
	t = T
	categories = set([])
	i = 0
	word = word + "$"
	while i < len(word):
		match = False
		for child in t.children:
			if child.character == "*":
				categories = categories.union(child.categories)
			elif child.character == word[i]:
				match = True
				t = child
				break
		if match == False:
			break
		else:
			i = i + 1
	if i == len(word):
		categories = categories.union(t.categories)
	return categories


ModuleNotFoundError: ignored

Socio-Linguistic Feature Extraction


Model


In [ ]:
# Tuned-Hyper Parameters
embed_size = 1024
hidden_size = 32
num_epochs=30
layer_size = 16
batch_size = 64

# 'Uni' : Unidirectional LSTM  |  'Bi' : Bidirectional LSTM  | 'TD' : Target-dependent LSTM
mode = 'Uni' 
augmentation = False

In [ ]:
# Function to group together candidate-words belonging to the same line.
def compress():
    lengths = []
    for i in range (1,len(ppd['left_context'])):
        # If left context contains only '<start>'
        if len(ppd['left_context'][i]) == 1:
            lengths.append(i)
    lengths.append(len(ppd['left_context']))
    compressor = []
    compressor.append(range(lengths[0]))
    for i in range (1,len(lengths)):
        compressor.append(range(lengths[i-1],lengths[i]))
    return compressor

In [ ]:
comp = compress()

In [ ]:
# Training Data, Test Data Preparation

def prep (train_indices, test_indices):
    print (len(train_indices), len(test_indices))

    # Ungroup training dataset candidate-words belonging to the same line.
    train_ids = []
    for i in range(len(train_indices)):
        train_ids.extend(comp[train_indices[i]])

    # Ungroup testing dataset candidate-words belonging to the same line.
    test_ids = []
    for i in range(len(test_indices)):
        test_ids.extend(comp[test_indices[i]])

    # Training Data Preparation
    train_left   = []
    train_right  = []
    train_middle = []
    train_labels = []
    train_pos_vec= []
    train_neg_vec= []
    train_liwc_vec=[]
    train_empath_vec=[]

    for id in train_ids:
        train_left.append(keras_left_context[id])
        train_right.append(keras_right_context[id])
        train_middle.append(keras_middle[id])
        train_labels.append(labels[id])
        train_pos_vec.append(pos_vec[id])
        train_neg_vec.append(ner_vec[id])
        train_liwc_vec.append(liwc_vec[id])
        train_empath_vec.append(empath_vec[id])

    train_left   = np.array(train_left)
    train_right  = np.array(train_right)
    train_middle = np.array(train_middle)
    train_labels = np.array(train_labels)
    train_middle = np.expand_dims(train_middle,axis=1)
    train_pos_vec= np.array(train_pos_vec)
    train_neg_vec= np.array(train_neg_vec)
    train_liwc_vec= np.array(train_liwc_vec)
    train_empath_vec= np.array(train_empath_vec)

    # Training Data Preparion
    val_left   = []
    val_right  = []
    val_middle = []
    val_labels = []
    val_pos_vec= []
    val_neg_vec= []
    val_liwc_vec=[]
    val_empath_vec=[]

    for id in test_ids:
        val_left.append(keras_left_context[id])
        val_right.append(keras_right_context[id])
        val_middle.append(keras_middle[id])
        val_labels.append(labels[id])
        val_pos_vec.append(pos_vec[id])
        val_ner_vec.append(ner_vec[id])
        val_liwc_vec.append(liwc_vec[id])
        val_empath_vec.append(empath_vec[id])

    val_left   = np.array(val_left)
    val_right  = np.array(val_right)
    val_middle = np.array(val_middle)
    val_labels = np.array(val_labels)
    val_middle = np.expand_dims(val_middle, axis=1)
    val_pos_vec=np.array(val_pos_vec)
    val_ner_vec=np.array(val_ner_vec)
    val_liwc_vec=np.array(val_liwc_vec)
    val_empath_vec=np.array(val_empath_vec)

    # Below part only for TD lstm
    if mode == 'TD':	    
    	train_left = np.concatenate((train_left, train_middle), axis=1)
	train_right = np.concatenate((train_middle, train_right), axis=1)
    	val_left = np.concatenate((val_left, val_middle), axis=1)
    	val_right = np.concatenate((val_middle, val_right), axis=1)
    return(train_left,train_right,train_middle,train_pos_vec,train_ner_vec,train_liwc_vec,train_empath_vec,train_labels,val_left,val_right,val_middle,val_pos_vec,val_neg_vec,val_liwc_vec,val_empath_vec,val_labels)



In [ ]:
# Dataset division for 3-fold cross validation
indices = list(range(len(comp)))
np.random.shuffle(indices)
bins = []
bins.append(indices[:int(0.33*len(indices))])
bins.append(indices[int(0.33*len(indices)):int(0.66*len(indices))])
bins.append(indices[int(0.66*len(indices)):])

In [ ]:
for i in range (3):
    print ("Fold {}".format(i+1))
    print (len(bins[0] + bins[1]),len(bins[2]))
    train_left,train_right,train_middle,pos_vec_train,ner_vec_train,liwc_vec_train,empath_vec_train,train_labels,val_left,val_right,val_middle,pos_vec_val,ner_vec_val,liwc_vec_val,empath_vec_val,val_labels = prep (bins[i%3] + bins[(i+1)%3], bins[(i+2)%3])
    Sar_model=model(train_left,train_right,train_middle,pos_vec_train,ner_vec_train,liwc_vec_train,empath_vec_train,train_labels,val_left,val_right,val_middle,pos_vec_val,ner_vec_val,liwc_vec_val,empath_vec_val,val_labels,bins[(i+2)%3])
    Sar_model.save_weights("Bert Tweets Aug.h5")
    print("Saved model to disk")